In [1]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

   ---------------------------------------- 0.0/536.6 kB ? eta -:--:--
   --------------------------------------- 536.6/536.6 kB 17.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   ---------------------------------------- 166.4/166.4 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.2.1
    Uninstalling datasets-2.2.1:
      Successfully uninstalled datasets-2.2.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
parlai 1.7.2 requires attrs~=20.2.0, but you have attrs 23.1.0 which is incompatible.
parlai 1.7.2 requires datasets<2.2.2,>=1.4.1, but you have datasets 2.17.0 which is incompatible.
parlai 1.7.2 requires fsspec~=2022.2.0, but you have fsspec 2023.10.0 which is incompatible.
parlai 1.7.2 requires protobuf<=3.20.3,>=3.8.0, but you have protobuf 4.21.12 which is incompatible.
parlai 1.7.2 requires tqdm~=4.62.1, but you have tqdm 4.66.1 which is incompatible.
rasa 3.6.10 requires attrs<22.2,>=19.3, but you have attrs 23.1.0 which is incompatible.
rasa 3.6.10 requires certifi>=2023.7.22, but you have certifi 2021.5.30 which is incompatible.
rasa 3.6.10 requires jsonschema<4.18,>=3.2, but you have jsonschema 4.20.0 which is incompatible.
rasa 3.6.10 requires networkx<2.7,>=2.4, but you have networkx 3.2.1 which is inc

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
parlai 1.7.2 requires attrs~=20.2.0, but you have attrs 23.1.0 which is incompatible.
parlai 1.7.2 requires datasets<2.2.2,>=1.4.1, but you have datasets 2.17.0 which is incompatible.
parlai 1.7.2 requires fsspec~=2022.2.0, but you have fsspec 2023.10.0 which is incompatible.
parlai 1.7.2 requires protobuf<=3.20.3,>=3.8.0, but you have protobuf 4.21.12 which is incompatible.
parlai 1.7.2 requires tqdm~=4.62.1, but you have tqdm 4.66.1 which is incompatible.
torchaudio 2.0.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torchdata==0.6.1, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.9 requires transformers<4.26.0,>=3.4.0, but you have transformers 4.27.2 which is incompatible.


In [16]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import evaluate
import time
import pandas as pd
import numpy as np

In [17]:
huggingface_dataset = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [18]:
modelname = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(modelname, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [19]:
def number_of_trainable_params(model):
  trainable_params = 0
  all_params = 0
  for _, param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  return f"Trainable Parameters : {trainable_params}\nAll Parameters : {all_params}"
  
print(number_of_trainable_params(original_model))


Trainable Parameters : 247577856
All Parameters : 247577856


In [20]:
index = 200

dialogue = dataset["test"][index]["dialogue"]
summary = dataset["test"][index]["summary"]

prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
"""
inputs = tokenizer(prompt, return_tensors="pt")
output = tokenizer.decode(
    original_model.generate(
        inputs['input_ids'],
        max_new_tokens=200,
    )[0],
    skip_special_tokens = True
)
seperation_line = '-'.join('' for x in range(100))
print(seperation_line)
print(f'INPUT PROMPT:\n{prompt}')
print(seperation_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(seperation_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

--------------------------------------------------------------------

In [21]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [22]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ['q', 'w'],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM 
)

In [23]:
peft_model = get_peft_model(original_model, lora_config)
print(number_of_trainable_params(peft_model))

Trainable Parameters : 1769472
All Parameters : 249347328


In [25]:
output_direc = f'./peft-dialogue-summary-trainaing-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_direc,
    auto_find_batch_size=True,
    learning_rate = 1e-3,
    num_train_epochs = 1,
    logging_steps = 1,
    max_steps = 1
)

peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    train_dataset = tokenized_datasets['train']
)

In [26]:
peft_trainer.train()
peft_model_path='./peft-dialogue-summary-checkpoint-local'
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

c:\Users\sanid\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

{'loss': 50.25, 'learning_rate': 0.0, 'epoch': 0.0}
{'train_runtime': 8125.3963, 'train_samples_per_second': 0.001, 'train_steps_per_second': 0.0, 'train_loss': 50.25, 'epoch': 0.0}


('./peft-dialogue-summary-checkpoint-local\\tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local\\special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local\\tokenizer.json')

In [36]:
!dir peft-dialogue-summary-checkpoint-local\adapter_model.bin


 Volume in drive D is New Volume
 Volume Serial Number is 58A7-18AA

 Directory of d:\Programming-1\Gen Ai\fine_tune_flan_t5\peft-dialogue-summary-checkpoint-local

03/23/2024  04:41 PM         7,103,049 adapter_model.bin
               1 File(s)      7,103,049 bytes
               0 Dir(s)  420,189,810,688 bytes free


In [37]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       './peft-dialogue-summary-checkpoint-local/', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [39]:
print(number_of_trainable_params(peft_model))

Trainable Parameters : 0
All Parameters : 249347328


In [40]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
# instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(seperation_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(seperation_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
# print(dash_line)
# print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(seperation_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person2: I'm thinking of upgrading your computer.
---------------------------------------------------------------------------------------------------
PEFT MODEL: #Person1#: I'm thinking of upgrading my computer.
